In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

data_all = pd.read_csv('../data/data.csv')

train, test = train_test_split(data_all, test_size=0.2, random_state=42)

X_train = train.drop(['CREDIT_SCORE','DEFAULT'], axis=1)
y_train = train['DEFAULT']

X_test = test.drop(['CREDIT_SCORE','DEFAULT'], axis=1)
y_test = test['DEFAULT']

In [10]:
from sklearn.base import BaseEstimator, TransformerMixin

class BaseColumnsSelector(BaseEstimator, TransformerMixin):
    def __init__(self, subset):
        self.subset = subset
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return X.loc[:, self.subset]
    
baseColumns = ['INCOME', 'SAVINGS', 'DEBT', 'T_CLOTHING_12', 'T_CLOTHING_6', 
       'T_EDUCATION_12', 'T_EDUCATION_6', 'T_ENTERTAINMENT_12',
       'T_ENTERTAINMENT_6', 'T_FINES_12',
       'T_FINES_6', 'T_GAMBLING_12', 'T_GAMBLING_6', 
       'T_GROCERIES_12', 'T_GROCERIES_6', 'T_HEALTH_12', 'T_HEALTH_6',
       'T_HOUSING_12', 'T_HOUSING_6', 'T_TAX_12', 'T_TAX_6', 'T_TRAVEL_12',
       'T_TRAVEL_6', 'R_TRAVEL', 'T_UTILITIES_12', 'T_UTILITIES_6', 
       'T_EXPENDITURE_12', 'T_EXPENDITURE_6', 
       'CAT_GAMBLING', 'CAT_CREDIT_CARD', 'CAT_MORTGAGE',
       'CAT_SAVINGS_ACCOUNT', 'CAT_DEPENDENTS']

groups = ['CLOTHING', 'EDUCATION', 'ENTERTAINMENT', 'FINES', 'GAMBLING', 'GROCERIES', 'HEALTH', 'HOUSING', 'TAX', 'TRAVEL', 'UTILITIES']
no_sense_in_division =['T_EDUCATION_12', 'T_FINES_12', 'T_GAMBLING_12', 'T_HOUSING_12', 'T_TAX_12', 'T_TRAVEL_12']
division_groups = ['CLOTHING',  'ENTERTAINMENT',  'GROCERIES', 'HEALTH',  'UTILITIES']


In [11]:
#być może zrobić fines_6 i fines_12 jako kategoryczne???? i gambling_6 i gambling_12 też
class OutliersReplacer(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        for column in X.columns:
            if column == 'SAVINGS':
                X.loc[X[column] > 2500000, column] = 2500000
            elif column == 'DEBT':
                X.loc[X[column] > 4000000, column] = 4000000
            elif column == 'T_CLOTHING_12':
                X.loc[X[column] > 32000, column] = 32000
            elif column == 'T_CLOTHING_6':
                X.loc[X[column] > 25000, column] = 25000
            elif column == 'T_HEALTH_12':
                X.loc[X[column] > 25000, column] = 25000
            elif column == 'T_HEALTH_6':
                X.loc[X[column] > 18000, column] = 18000
            elif column == 'T_TRAVEL_12':
                X.loc[X[column] > 150000, column] = 150000
            elif column == 'T_TRAVEL_6':
                X.loc[X[column] > 110000, column] = 110000
        return X

In [12]:
class RatioColumnsAdder(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        for idx, row in X.iterrows():
            #DEBT/INCOME
            if row['INCOME'] == 0:
                X.at[idx, 'R_DEBT_INCOME'] = None
            else:
                X.at[idx, 'R_DEBT_INCOME'] = row['DEBT'] / row['INCOME']
            #SAVINGS/INCOME
            if row['INCOME'] == 0:
                X.at[idx, 'R_SAVINGS_INCOME'] = None
            else:
                X.at[idx, 'R_SAVINGS_INCOME'] = row['SAVINGS'] / row['INCOME']
            #DEBT/SAVINGS
            if row['SAVINGS'] == 0:
                X.at[idx, 'R_DEBT_SAVINGS'] = None
            else:
                X.at[idx, 'R_DEBT_SAVINGS'] = row['DEBT'] / row['SAVINGS']
            for group in division_groups:
                if row['T_' + group + '_12'] == 0:
                    X.at[idx, 'R_' + group] = None
                else:
                    X.at[idx, 'R_' + group] = row['T_' + group + '_6'] / row['T_' + group + '_12']
            for group in groups:
                if row['INCOME'] == 0:
                    X.at[idx, 'R_'+group+'_INCOME'] = None
                else:
                    X.at[idx, 'R_'+group+'_INCOME'] = row['T_'+group+'_6'] / row['INCOME']
                    
                if row['SAVINGS'] == 0:
                    X.at[idx, 'R_'+group+'_SAVINGS'] = None
                else:
                    X.at[idx, 'R_'+group+'_SAVINGS'] = row['T_'+group+'_6'] / row['SAVINGS']

                if row['DEBT'] == 0:
                    X.at[idx, 'R_'+group+'_DEBT'] = None
                else:
                    X.at[idx, 'R_'+group+'_DEBT'] = row['T_'+group+'_6'] / row['DEBT']
        return X

In [13]:
class DropColumns(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        potentialColumnsToDrop = ['T_EDUCATION_12', 'T_FINES_12', 'T_GAMBLING_12', 'T_HOUSING_12', 'T_TAX_12', 'T_TRAVEL_12', 'T_EDUCATION_6','T_ENTERTAINMENT_6','T_GAMBLING_6','T_GROCERIES_6','T_HOUSING_6','T_EXPENDITURE_12', 'T_EXPENDITURE_6','R_GROCERIES_DEBT', 'INCOME', 'T_UTILITIES_6', 'R_EDUCATION_DEBT', 'T_UTILITIES_12', 'R_CLOTHING_DEBT',
        'CAT_DEPENDENTS', 'R_ENTERTAINMENT_SAVINGS', 'R_FINES_INCOME',
        'R_FINES_SAVINGS', 'R_FINES_DEBT', 'R_GROCERIES_SAVINGS',
        'CAT_SAVINGS_ACCOUNT', 'R_HOUSING_INCOME', 'R_TAX_INCOME',
        'R_TAX_SAVINGS', 'R_TRAVEL_DEBT', 'R_UTILITIES_DEBT', 'CAT_GAMBLING',
        'CAT_DEBT', 'CAT_MORTGAGE', 'SAVINGS', 'R_UTILITIES_SAVINGS', 'R_EDUCATION', 'R_FINES', 'R_GAMBLING', 'R_HOUSING', 'R_GROCERIES_INCOME', 'T_ENTERTAINMENT_12', 'R_ENTERTAINMENT',
       'R_TRAVEL_SAVINGS', 'R_GAMBLING_SAVINGS', 'T_CLOTHING_6']
        for column in potentialColumnsToDrop:
            if column in X.columns:
                X.drop(column, axis=1, inplace=True)
        return X

In [14]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

class MissingValuesFiller(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        numerical_columns = X.select_dtypes(include=[np.number]).columns
        imputer = KNNImputer(n_neighbors=7)
        X[numerical_columns] = imputer.fit_transform(X[numerical_columns])
        return X

In [15]:
#best parameters :
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
#SupportVectorClassifier
from sklearn.svm import SVC as SupportVectorClassifier
#standart scaler
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

for k in ['linear', 'poly', 'rbf', 'sigmoid']:
    for c in [0.1, 1, 10, 100]:
            pipeline = Pipeline([
                ('selector', BaseColumnsSelector(baseColumns)),
                ('outliers_replacer', OutliersReplacer()),
                ('ratio_columns_adder', RatioColumnsAdder()),
                ('missing_values_filler', MissingValuesFiller()),
                ('drop_columns', DropColumns()),
                ('scaler', StandardScaler()),
                ('classifier', SupportVectorClassifier(kernel = k, C = c))
            ])
            kfold = KFold(n_splits=5, random_state=42, shuffle=True)
            score_acc = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring='accuracy')
            score_recall = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring='recall')
            score_precision = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring='precision')
            score_f1 = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring='f1')
            print(f"kernel: {k}, C: {c}, accuracy: {score_acc.mean()}, recall: {score_recall.mean()}, precision: {score_precision.mean()}, f1: {score_f1.mean()}")

kernel: linear, C: 0.1, accuracy: 0.734375, recall: 0.12042624042624044, precision: 0.666883116883117, f1: 0.19746475746475747
kernel: linear, C: 1, accuracy: 0.7359375, recall: 0.1491042291042291, precision: 0.667142857142857, f1: 0.2320472132230041
kernel: linear, C: 10, accuracy: 0.7359375, recall: 0.1491042291042291, precision: 0.667142857142857, f1: 0.2320472132230041


KeyboardInterrupt: 

In [15]:
#best parameters :
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
#SupportVectorClassifier
from sklearn.svm import SVC as SupportVectorClassifier
#standart scaler
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

for k in [ 'poly', 'rbf', 'sigmoid']:
    for c in [0.1, 1, 10, 100]:
            pipeline = Pipeline([
                ('selector', BaseColumnsSelector(baseColumns)),
                ('outliers_replacer', OutliersReplacer()),
                ('ratio_columns_adder', RatioColumnsAdder()),
                ('missing_values_filler', MissingValuesFiller()),
                ('drop_columns', DropColumns()),
                ('scaler', StandardScaler()),
                ('classifier', SupportVectorClassifier(kernel = k, C = c))
            ])
            kfold = KFold(n_splits=5, random_state=42, shuffle=True)
            score_acc = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring='accuracy')
            score_recall = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring='recall')
            score_precision = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring='precision')
            score_f1 = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring='f1')
            print(f"kernel: {k}, C: {c}, accuracy: {score_acc.mean()}, recall: {score_recall.mean()}, precision: {score_precision.mean()}, f1: {score_f1.mean()}")

kernel: poly, C: 0.1, accuracy: 0.7234375, recall: 0.044062604062604066, precision: 0.7, f1: 0.08223938223938224
kernel: poly, C: 1, accuracy: 0.7296875, recall: 0.12496836496836497, precision: 0.5554545454545454, f1: 0.19914529914529916
kernel: poly, C: 10, accuracy: 0.6953125, recall: 0.22895770895770892, precision: 0.42330827067669174, f1: 0.29477436907777466
kernel: poly, C: 100, accuracy: 0.6171875, recall: 0.31219447219447216, precision: 0.3225384024577573, f1: 0.31453584240335414


/Users/katebokhan/anaconda3/envs/6.86x/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/katebokhan/anaconda3/envs/6.86x/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/katebokhan/anaconda3/envs/6.86x/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/katebokhan/anaconda3/envs/6.86x/lib/pyt

kernel: rbf, C: 0.1, accuracy: 0.7171875, recall: 0.0, precision: 0.0, f1: 0.0
kernel: rbf, C: 1, accuracy: 0.734375, recall: 0.10865134865134865, precision: 0.6995238095238095, f1: 0.1817450603252317
kernel: rbf, C: 10, accuracy: 0.665625, recall: 0.24083916083916082, precision: 0.35192000981474664, f1: 0.2807726787376307
kernel: rbf, C: 100, accuracy: 0.5921875, recall: 0.28211788211788213, precision: 0.2793248860431195, f1: 0.2746529388000197


/Users/katebokhan/anaconda3/envs/6.86x/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/katebokhan/anaconda3/envs/6.86x/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/katebokhan/anaconda3/envs/6.86x/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/katebokhan/anaconda3/envs/6.86x/lib/pyt

kernel: sigmoid, C: 0.1, accuracy: 0.7171875, recall: 0.0, precision: 0.0, f1: 0.0
kernel: sigmoid, C: 1, accuracy: 0.7125, recall: 0.16916416916416915, precision: 0.4635599694423223, f1: 0.24279598662207355
kernel: sigmoid, C: 10, accuracy: 0.6359375, recall: 0.3033366633366633, precision: 0.32797821350762524, f1: 0.31285672645233736
kernel: sigmoid, C: 100, accuracy: 0.6171875, recall: 0.32947052947052946, precision: 0.29482631189948266, f1: 0.31013192087234387


In [17]:
pipeline = Pipeline([
    ('selector', BaseColumnsSelector(baseColumns)),
    ('outliers_replacer', OutliersReplacer()),
    ('ratio_columns_adder', RatioColumnsAdder()),
    ('missing_values_filler', MissingValuesFiller()),
    ('drop_columns', DropColumns()),
    ('scaler', StandardScaler()),
    ('classifier', SupportVectorClassifier(kernel = 'linear', C = 10))
    ])
kfold = KFold(n_splits=5, random_state=42, shuffle=True)
#test full report
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))


[[111   3]
 [ 36  10]]
              precision    recall  f1-score   support

           0       0.76      0.97      0.85       114
           1       0.77      0.22      0.34        46

    accuracy                           0.76       160
   macro avg       0.76      0.60      0.59       160
weighted avg       0.76      0.76      0.70       160

0.75625
